In [5]:
import torch
import os
import numpy as np
import soundfile as sf
base_path = r"D:\\work\\speechEnhancement\\datasets\\voicebank_demand"
audio_name = "p232_001.wav"
noisy_audio_path = os.path.join(base_path, "noisy_testset_wav", audio_name)

feat_wav, _ = sf.read(noisy_audio_path)  # (27861,)

c = np.sqrt(len(feat_wav) / np.sum((feat_wav ** 2.0)))
feat_wav = feat_wav * c   # (27861,)
wav_len = len(feat_wav)   # wav_len = 27861
frame_num = int(np.ceil((wav_len - 320 + 320) / 160 + 1))  # 176
fake_wav_len = (frame_num - 1) * 160 + 320 - 320    # 28000
left_sample = fake_wav_len - wav_len  # 139 补零
feat_wav = torch.FloatTensor(np.concatenate((feat_wav, np.zeros([left_sample])), axis=0)) # (28000,)
feat_x = torch.stft(feat_wav.unsqueeze(dim=0), n_fft=320, hop_length=160, win_length=320,
                    window=torch.hann_window(320), return_complex=True).permute(0, 2, 1)

#compressed
feat_mag, feat_phase = torch.norm(feat_x, dim=1)**0.5, torch.angle(feat_x)
feat_x_compress = torch.stack((feat_mag*torch.cos(feat_phase), feat_mag*torch.sin(feat_phase)), dim=1)

In [7]:
import torchaudio
feat_wav, _ = torchaudio.load(noisy_audio_path)  # (27861,)
feat_wav = feat_wav.squeeze(0)
c = torch.sqrt(len(feat_wav) / torch.sum((feat_wav ** 2.0)))
feat_wav = feat_wav * c   # (27861,)
wav_len = len(feat_wav)   # wav_len = 27861
frame_num = int(np.ceil((wav_len - 320 + 320) / 160 + 1))  # 176
fake_wav_len = (frame_num - 1) * 160 + 320 - 320    # 28000
left_sample = fake_wav_len - wav_len  # 139 补零
feat_wav = (torch.concatenate((feat_wav, torch.zeros([left_sample])), dim=0)) # (28000,)
feat_x = torch.stft(feat_wav.unsqueeze(dim=0), n_fft=320, hop_length=160, win_length=320,
                    window=torch.hann_window(320), return_complex=True).permute(0, 2, 1)

#compressed
feat_mag, feat_phase = torch.norm(feat_x, dim=1)**0.5, torch.angle(feat_x)
feat_x_compress1 = torch.stack((feat_mag*torch.cos(feat_phase), feat_mag*torch.sin(feat_phase)), dim=1)

In [8]:
print(torch.sum(torch.abs(feat_x_compress1-feat_x_compress)))

tensor(0.5544)
